In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display
from datetime import date, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'
import matplotlib.pyplot as plt
from matplotlib import colors

import ptm_lib as ptm
%load_ext autoreload
%autoreload 2

# import pandas_datareader as pdr
# from pandas_datareader import wb
# import quandl


In [ ]:
def Dist_Of_Rets(df): 
    col = df.columns[0]
    df[col+' MoM'] = df[col]/df[col].shift(1)-1  # MoM %
    df[col+' YoY'] = df[col]/df[col].shift(12)-1 # YoY %
    
    DoR, summary_stats, boundary_stats = ticker_stats(pd.DataFrame(df[col+' MoM']), 20)    
    print("\n ")
    print(DoR)
    print("\n ")
    print(summary_stats)
    print("\n ")
    print(boundary_stats)
    
#     DoR, summary_stats, boundary_stats = ticker_stats(df[col+' YoY'], 20)    
#     print("\n ")
#     print(DoR)
#     print("\n ")
#     print(summary_stats)
#     print("\n ")
#     print(boundary_stats)

    posneg_plot(pd.DataFrame(df[col+' MoM']))
    print("\n ")
    print(df)
    

def dist_plot(DoR, df, bin_lims):    # Plotting
    ser = df[df.columns[0]]
    fig, axs = plt.subplots(1, 1, figsize =(12, 5), tight_layout = True)   # Creating histogram
    # Add padding between axes and labels 
    axs.xaxis.set_tick_params(pad = 5) 
    axs.yaxis.set_tick_params(pad = 5) 
    axs.grid(visible = True, color ='grey', linestyle ='-.', linewidth = 0.5, alpha = 0.6)   # Add x, y gridlines 
    N, bins, patches = axs.hist(ser, bins = bin_lims)   # Creating histogram
    
    axs.plot(ser[-1], 0, 'ro')  # # Add a point representing the latest value 'ro' means red circles
    rank = (ser < ser[-1]).sum() / len(ser)
    indx = df.index[-1].date()
    txt = (indx.strftime('%d-%b-%y')+" "+"{:.5f}".format(ser[-1])+" %rank:"+"{: .2f}".format(rank))
    axs.annotate(txt, (ser[-1],0), xytext=(ser[-1],+15), arrowprops=dict(arrowstyle="->",facecolor='red'),color='red' )

    
    # Setting color
    fracs = ((N**(1 / 5)) / N.max())
    norm = colors.Normalize(fracs.min(), fracs.max())

    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    plt.xticks(DoR['Bins'], DoR['Range'], rotation=45, ha='right')  # Rotate x-labels by 45 degrees
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.title('Distribution of Returns '+df.columns[0])
    plt.show()
    
def posneg_plot(df):
    desc = df.columns[0]
    df['pos'] = np.maximum(df[desc], 0)
    df['neg'] = np.minimum(df[desc], 0)

    # Define the range where y > 0 and y < 0
    y_above_zero = np.maximum(df[desc], 0)
    y_below_zero = np.minimum(df[desc], 0)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=df['pos'], name='positive', fill='tozeroy')) # fill down to xaxis
    fig.add_trace(go.Scatter(x=df.index, y=df['neg'], name='negative', fill='tozeroy')) # fill to trace0 y

    # fig = px.area(df, x=df.index,y='Spread')
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=5, label="5y", step="year", stepmode="backward"),
                dict(count=10, label="10y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.update_layout(title_text='<b>'+df.columns[0]+'<b>', title_x=0.5)
    fig.update_layout(margin=dict(l=0, r=0, t=25, b=0))
    fig.show()


In [ ]:
def ticker_stats(df, num_bins):
    ser = df[df.columns[0]]
    mean_Ret = ser.mean()
    std_Ret = ser.std()

    DoR = pd.DataFrame()

    do_lim = mean_Ret-3*std_Ret ; up_lim = mean_Ret+3*std_Ret
    bin_lims = np.linspace(do_lim, up_lim, num=num_bins, endpoint=True)
    DoR['Bins'] = bin_lims
    DoR['Frequency'] = np.nan
    DoR['Range'] = " "
    DoR['Probability'] = np.nan
    DoR['Cum. Prob.'] = np.nan

    for i,val in enumerate(DoR['Bins']):
        if i==0:   # Count Frequency
            DoR.loc[i,['Frequency']] = len(ser[ser <= bin_lims[i]])
            DoR.loc[i,['Range']] = "Less than "+str(round(val*100,3))+"%"
        elif i==len(bin_lims)-1:
            DoR.loc[i,['Frequency']] = len(ser[ser > bin_lims[i]])
            DoR.loc[i,['Range']] = "More than "+str(round(val*100,3))+"%"
        else:
            DoR.loc[i,['Frequency']] = len(ser[(ser > bin_lims[i-1]) & (ser <= bin_lims[i])])
            DoR.loc[i,['Range']] = str(round(DoR['Bins'][i-1]*100,3))+"%" + " to " + str(round(val*100,3))+"%"

        DoR.loc[i,['Probability']] = 100*DoR['Frequency'][i]/len(ser)   # Calculate Probability
        if i==0:   # Calculate Cumulative Probability
            DoR.loc[i,['Cum. Prob.']] = DoR['Probability'][i]
        else:
            DoR.loc[i,['Cum. Prob.']] = DoR['Probability'][i]+DoR['Cum. Prob.'][i-1]

    
    # Summary of Pos and Neg Returns
    posneg_rets = pd.DataFrame({'Desc': ['Positive Rets', 'Negative Rets', 'Zero'],
                       'Av Returns': "",'Count': "",'Frequency': "",'Freq. Adj Ret': ""})
    total_counts = len(ser)
    for i in range(len(posneg_rets)):
        if i==0:
            pos_vals = ser[(ser > 0)]
            posneg_rets.loc[i,['Av Returns']] = pos_vals.mean()
            posneg_rets.loc[i,['Count']] = pos_vals.count()
        elif i==1:
            neg_vals = ser[(ser < 0)]
            posneg_rets.loc[i,['Av Returns']] = neg_vals.mean()
            posneg_rets.loc[i,['Count']] = neg_vals.count()
        else:
            zero_vals = ser[(ser == 0)]
            posneg_rets.loc[i,['Av Returns']] = zero_vals.mean()
            posneg_rets.loc[i,['Count']] = zero_vals.count()    
        posneg_rets.loc[i,['Frequency']] = 100*posneg_rets['Count'][i]/total_counts
        posneg_rets.loc[i,['Freq. Adj Ret']] = posneg_rets['Av Returns'][i]*posneg_rets['Frequency'][i]

    
    # Summary of Boundaries
    stdDev_summary = pd.DataFrame({'StdDev': [1,2,3],'UpperB': "",'LowerB': "",'Count': "",'% Count': ""})
    stdDev_summary.set_index('StdDev')
    for i in range(1,len(stdDev_summary)+1):
        stdDev_summary.loc[i-1,['UpperB']] = mean_Ret+i*std_Ret
        stdDev_summary.loc[i-1,['LowerB']] = mean_Ret-i*std_Ret
        stdDev_summary.loc[i-1,['Count']] = len(ser[(ser > stdDev_summary['LowerB'][i-1]) & (ser < stdDev_summary['UpperB'][i-1])])
        stdDev_summary.loc[i-1,['% Count']] = 100*stdDev_summary['Count'][i-1]/total_counts
    
    dist_plot(DoR, df, bin_lims)
    print("\n Descriptive Statistics")
    print(ser.describe())
    return DoR, posneg_rets, stdDev_summary


In [ ]:
from_date = date(1928, 1, 1) 
today = date.today()
# Money Supply
tickers = ['M2SL']
file = 'M2'
frequency = 'm'

data = ptm.loadNupdate(tickers, from_date, today, frequency, file)
Dist_Of_Rets(data)